In [10]:
import os
import pandas as pd
import numpy as np
from helpers import *
from PIL import Image, ImageDraw
from time import time
from openslide import OpenSlide

In [2]:
os.getcwd()

'/home/user/Documentos/WSI/ipynbs'

### vamos a reconstruir la mask para 'I-2021-4001 A3'

In [6]:
#dataframe con las coordenadas de los patch de la wsi
sel_file = 'I-2021-4001 A3'
df = pd.read_parquet('/home/user/Documentos/WSI/dataset/coords.parquet')
df = df[df.file == sel_file]
df

,file,x,y
0,I-2021-4001 A3,52736,1536
1,I-2021-4001 A3,53248,1536
2,I-2021-4001 A3,52736,2048
3,I-2021-4001 A3,53248,2048
4,I-2021-4001 A3,52736,2560
...,...,...,...
209,I-2021-4001 A3,30208,13824
210,I-2021-4001 A3,30720,13824
211,I-2021-4001 A3,31232,13824
212,I-2021-4001 A3,31744,13824


In [14]:
#lista con los poligonos(anotaciones) de interes
d = get_vert(os.path.join('/home/user/Documentos/WSI/WSI',sel_file+'.xml'))
type(d),len(d), len(d[0])

(list, 18, 553)

función helper: bounding box

In [20]:
#bounding box para las anotaciones del xml original 
def get_bound_box(pols_list):
    min_x, max_x, min_y, max_y = [], [], [], []
    for pol in pols_list:
        min_x.append(min(pol, key=lambda p: p[0])[0])
        max_x.append(max(pol, key=lambda p: p[0])[0])
        min_y.append(min(pol, key=lambda p: p[1])[1])
        max_y.append(max(pol, key=lambda p: p[1])[1])
        
    print(f'{min(min_x)} < X < {max(max_x)}\n{min(min_y)} < Y < {max(max_y)} ')
        
get_bound_box(d)

449 < X < 8240
5020 < Y < 13945 


In [48]:
#buscamos las filas (ids) que estén en la bounding box
ids_df = df[(df.x > 400) & (df.x < 8300)]
ids_df.head()

,file,x,y
7,I-2021-4001 A3,5120,3072
10,I-2021-4001 A3,5120,3584
14,I-2021-4001 A3,4608,4096
15,I-2021-4001 A3,5120,4096
20,I-2021-4001 A3,4608,4608


In [51]:
#ids_df.iloc[7].x
#for i, (_, row) in enumerate(ids_df.iterrows()):
#    print(i, row.x, row.y)

In [22]:
ids_df.index

Index([  7,  10,  14,  15,  20,  25,  26,  31,  32,  38,  39,  44,  45,  50,
        51,  56,  57,  58,  65,  66,  75,  76,  77,  86,  87,  96,  97, 107,
       108, 118, 133, 144, 145, 146, 148, 149, 150, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 183, 184, 185, 186, 187, 188, 189, 190, 191],
      dtype='int64')

In [29]:
#ahora sabemos que masks leer, asumiendo que el fondo debe ser negro
ids = [f'{x:05d}' for x in ids_df.index]
ids[-7:]

['00185', '00186', '00187', '00188', '00189', '00190', '00191']

In [41]:
#masks relevantes
path = '/home/user/Documentos/WSI/dataset/mask'
files = os.listdir(path)
files = [x for x in files if x.split('_')[0] in ids]
files = sorted(files)
files[:3]

['00007_2579_1548.png', '00010_2579_1806.png', '00014_2322_2064.png']

In [50]:
large_image = Image.new(size=(8500,15000),color=0, mode='1') #size basado en la bounding box

for i, (_, row) in enumerate(ids_df.iterrows()):
    small_image = Image.open(os.path.join(path, files[i])) #leemos la mask 
    large_image.paste(small_image, (row.x, row.y))         #pegamos sobre fondo negro

large_image.show("result_image.png")                       #mostrar

Abriendo en una sesión existente del navegador


[13108:13108:0100/000000.612642:ERROR:zygote_linux.cc(662)] write: Tubería rota (32)


En efecto, se genera una mask muy similar a la original. Aparenta perder información, pero en la zona donde aquello sucede, la anotación incluye mucho fondo.

Al generar el dataset con un valor de $pix$ menor (actual es $512$) la perdida debería ser menor...

----

## Reconstruir any mask from the parched_mask in dataset

In [70]:
#must give just the filename, not .svs or .xml included
def all_(sel_file):
    d = get_vert(os.path.join('/home/user/Documentos/WSI/WSI',sel_file+'.xml'))
    get_bound_box(d)
    x_min = int(input('Ingrese x_min: '))
    x_max = int(input('Ingrese x_max: '))
    y_min = int(input('Ingrese y_min: '))
    y_max = int(input('Ingrese y_max: '))
    df = pd.read_parquet('/home/user/Documentos/WSI/dataset/coords.parquet')
    df = df[df.file == sel_file]
    ids_df = df[(df.x > x_min) & (df.x < x_max)]
    ids = [f'{x:05d}' for x in ids_df.index]
    files = os.listdir('/home/user/Documentos/WSI/dataset/mask')
    files = sorted([x for x in files if x.split('_')[0] in ids])
    #large_image = Image.new(size=(x_max, y_max),color=0, mode='1') 
    large_image = Image.new(size=(x_max-x_min, y_max-y_min),color=0, mode='1') 

    for i, (_, row) in enumerate(ids_df.iterrows()):
        small_image = Image.open(os.path.join('/home/user/Documentos/WSI/dataset/mask', files[i])) 
        #large_image.paste(small_image, (row.x, row.y))
        large_image.paste(small_image, (row.x-x_min, row.y-y_min))

    large_image.show()

In [60]:
filenames('/home/user/Documentos/WSI/WSI', 'xml')

['I-2021-4001 A3.xml',
 'I-2021-4001 A4.xml',
 'I-2021-5616 A1.xml',
 'I-2021-5616 A2.xml',
 'I-2021-5616 A3.xml',
 'I-2021-5616 A4.xml',
 'I-2021-5616 B6.xml']

In [71]:
all_('I-2021-4001 A4')

25270 < X < 30260
9025 < Y < 15832 
Ingrese x_min: 25000
Ingrese x_max: 30000
Ingrese y_min: 9000
Ingrese y_max: 16000
Abriendo en una sesión existente del navegador


[14455:14455:0100/000000.064411:ERROR:zygote_linux.cc(662)] write: Tubería rota (32)


---

## plotting the mask over the patch

In [4]:
#def coord_to_um(filename):
#    p = filename.split('.')[0].split('_')
#    return int(int(p[1])/0.5038), int(int(p[2])/0.5038)
#coord_to_um(file)

In [5]:
#file = '00025_2064_2579.png'
#file = '00057_1290_3869.png'
#file = '01001_37402_5417.png'
#file = '01238_35081_2322.png'
#file = '01012_37918_5675.png'
path = '/home/user/Documentos/WSI/dataset'
image = Image.open(os.path.join(path,'patch',file)).convert("RGBA")
mask = np.array(Image.open(os.path.join(path,'mask',file)).convert("L"))
df = pd.read_parquet('/home/user/Documentos/WSI/dataset/coords.parquet')
df.iloc[57] #escribir aqui el id del file deseado

file    I-2021-4001 A3
x                 2560
y                 7680
Name: 57, dtype: object

In [6]:
# Adjust the alpha channel of the mask to make it translucent
alpha = 120  # Set the alpha value (0-255)  #120-180 buenos valores
mask_with_alpha = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)
mask_with_alpha[:, :, 0] = 255  # Red channel
mask_with_alpha[:, :, 1] = 255  # Green channel
mask_with_alpha[:, :, 2] = 255  # Blue channel
mask_with_alpha[:, :, 3] = mask * alpha  # Alpha channel

# Create a light blue color (RGBA)
light_blue = (173, 216, 230, 255)  # RGB: (173, 216, 230), Alpha: 255
# Create a new image with the same size as the original image and fill it with light blue
overlay = Image.new("RGBA", image.size, light_blue)
# Create a composite image by blending the original image and the overlay using the mask
composite = Image.composite(overlay, image, Image.fromarray(mask_with_alpha))

composite.convert('RGB').show()

Abriendo en una sesión existente del navegador


[3856:3856:0100/000000.748612:ERROR:zygote_linux.cc(662)] write: Tubería rota (32)


---

## reconstructing WSI

In [12]:
path = '/home/user/Documentos/UFRO UNICAMP Images'
files = filenames(path, 'svs')
dims_by_filename = {file[:-4]: OpenSlide(os.path.join(path,file)).dimensions for file in files}

In [23]:
#input: nombre del archivo de interes (sin svs/xml)
#       path a la carpeta dataset (la que contiene patch+mask)
#output:  carpeta con imagenes de los pares patch+mask sobrepuestos

# crítica: ¿ mejora la performance si chequeo si mask == black ?

def save_all_masked_patches(filename, path = '/home/user/Documentos/WSI/dataset'):
    df = pd.read_parquet(os.path.join(path,'coords.parquet'))
    df = df[df.file == filename]
    ids = [f'{x:05d}' for x in df.index]
    files = os.listdir(os.path.join(path,'mask'))
    files = sorted([x for x in files if x.split('_')[0] in ids])
    for file in files:
        image = Image.open(os.path.join(path,'patch',file)).convert("RGBA")
        mask = np.array(Image.open(os.path.join(path,'mask',file)).convert("L"))
        alpha = 120  # Set the alpha value (0-255)  #120-180 buenos valores
        mask_with_alpha = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)
        mask_with_alpha[:, :, 0] = 255  # Red channel
        mask_with_alpha[:, :, 1] = 255  # Green channel
        mask_with_alpha[:, :, 2] = 255  # Blue channel
        mask_with_alpha[:, :, 3] = mask * alpha  # Alpha channel
        light_blue = (173, 216, 230, 255)  # RGB: (173, 216, 230), Alpha: 255
        overlay = Image.new("RGBA", image.size, light_blue)
        composite = Image.composite(overlay, image, Image.fromarray(mask_with_alpha))
        check_dir(os.path.join('/home/user/Documentos/WSI','temp',filename))
        composite.convert('RGB').save(os.path.join('/home/user/Documentos/WSI','temp',filename,file))

fn = 'I-2021-4001 A4'
save_all_masked_patches(fn)

'''
for i, (_, row) in enumerate(ids_df.iterrows()):
    small_image = Image.open(os.path.join(path, files[i])) #leemos la mask 
    large_image.paste(small_image, (row.x, row.y))         #pegamos sobre fondo negro

for patch_image, top_left_x, top_left_y in patches:
    patch = Image.open(os.path.join(path,file))
    full_image.paste(patch, (top_left_x, top_left_y))
'''

'\nfor i, (_, row) in enumerate(ids_df.iterrows()):\n    small_image = Image.open(os.path.join(path, files[i])) #leemos la mask \n    large_image.paste(small_image, (row.x, row.y))         #pegamos sobre fondo negro\n\nfor patch_image, top_left_x, top_left_y in patches:\n    patch = Image.open(os.path.join(path,file))\n    full_image.paste(patch, (top_left_x, top_left_y))\n'

In [21]:
def reconstruct_wsi(filename, path_masked_patches, path_coords_df, full_image_size):
    full_image = Image.new('RGB', full_image_size, (240,240,240))
    df = pd.read_parquet(path_coords_df)
    df = df[df.file == filename]
    files = sorted(os.listdir(path_masked_patches))
    #ids = [int(x.split('.')[0].split('_')[0]) for x in files]
    
    for i, (_, row) in enumerate(df.iterrows()):
        patch = Image.open(os.path.join(path_masked_patches, files[i])) #leemos la mask 
        full_image.paste(patch, (row.x, row.y))         #pegamos sobre fondo 

    for patch_image, top_left_x, top_left_y in patches:
        patch = Image.open(os.path.join(path,file))
        full_image.paste(patch, (top_left_x, top_left_y))

    return sorted(files)

fn = 'I-2021-4001 A3'
path_masked_patches = '/home/user/Documentos/WSI/temp/I-2021-4001 A3'
path_coords_df = '/home/user/Documentos/WSI/dataset/coords.parquet'
reconstruct_wsi(fn, path_masked_patches, path_coords_df, dims_by_filename[fn])

['00000_26568_774.png',
 '00001_26826_774.png',
 '00002_26568_1032.png',
 '00003_26826_1032.png',
 '00004_26568_1290.png',
 '00005_26826_1290.png',
 '00006_27084_1290.png',
 '00007_2579_1548.png',
 '00008_26568_1548.png',
 '00009_26826_1548.png',
 '00010_2579_1806.png',
 '00011_14961_1806.png',
 '00012_26568_1806.png',
 '00013_26826_1806.png',
 '00014_2322_2064.png',
 '00015_2579_2064.png',
 '00016_14703_2064.png',
 '00017_14961_2064.png',
 '00018_26568_2064.png',
 '00019_26826_2064.png',
 '00020_2322_2322.png',
 '00021_14703_2322.png',
 '00022_14961_2322.png',
 '00023_26568_2322.png',
 '00024_26826_2322.png',
 '00025_2064_2579.png',
 '00026_2322_2579.png',
 '00027_14445_2579.png',
 '00028_14703_2579.png',
 '00029_26568_2579.png',
 '00030_26826_2579.png',
 '00031_1806_2837.png',
 '00032_2064_2837.png',
 '00033_14445_2837.png',
 '00034_14703_2837.png',
 '00035_26310_2837.png',
 '00036_26568_2837.png',
 '00037_26826_2837.png',
 '00038_1806_3095.png',
 '00039_2064_3095.png',
 '00040_14187

int

In [ ]:
def reconstruct_wsi(patches, full_image_size):
    # Create a new blank image with the full size
    full_image = Image.new('RGB', full_image_size, (240,240,240))
    
    path = '/home/user/Documentos/WSI/temp'
    # Paste each patch onto the full image
    for i, (_, row) in enumerate(ids_df.iterrows()):
    small_image = Image.open(os.path.join(path, files[i])) #leemos la mask 
    large_image.paste(small_image, (row.x, row.y))         #pegamos sobre fondo negro

    for patch_image, top_left_x, top_left_y in patches:
        patch = Image.open(os.path.join(path,file))
        full_image.paste(patch, (top_left_x, top_left_y))

    return full_image

# Usage example:
# Assuming patches is a list of tuples (patch_image, top_left_x, top_left_y)
# and full_image_size contains the dimensions of the full image (width, height)
full_image = reconstruct_full_image_with_pil(patches, full_image_size)
full_image.show()